In [ ]:
import cv2
import dlib
import numpy as np

In [ ]:
# Function to return the landmark coordinated in numpy array format
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
    # loop over the 68 facial landmarks and convert them to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords
# To add the eye region ot the mask
def eye_on_mask(mask, side):
    # selecting the list containing points in the side array
    points = [shape[i] for i in side]
    points = np.array(points, dtype=np.int32)
    # This function makes a convex polygon shape with the given points and sets those coordinate values in mask to 255
    mask = cv2.fillConvexPoly(mask, points, 255)
    return mask
# To draw the red color dot at the center of eyeball
def contouring(thresh, mid, img, right=False):
    # This returns the contours ie. areas with similar colour texture 
    cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    try:
        cnt = max(cnts, key = cv2.contourArea) # finding the contour with max are ie. the eyeball
        # The folowwing lines are used to detect the movements of the contour area
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        # To ensure midpoint stays in same area to remove random detections or objects other than eyeball
        if right:
            cx += mid
        cv2.circle(img, (cx, cy), 4, (0, 0, 255), 2)
    except:
        pass

In [ ]:
# These are used to get the face locations in 68 points using deep learning library dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# These are the numbers of the points in the face where left and right eye is matched by dlib 
left = [36, 37, 38, 39, 40, 41]
right = [42, 43, 44, 45, 46, 47]

cap = cv2.VideoCapture(0)
ret, img = cap.read()
thresh = img.copy()

cv2.namedWindow('image')
# The smaller the kernel more stricter the detection more the chances of false negatives
kernel = np.ones((9, 9), np.uint8)

def nothing(x):
    pass
# Uncomment the following line for gui application to manually select the threshold
#cv2.createTrackbar('threshold', 'image', 0, 255, nothing)

In [ ]:
# The infinite loop for video capture
while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detects the region of interest where face is found
    rects = detector(gray, 1)
    for rect in rects:
        # For each region where face is found, the following lines predict the 68 locations corresponding to each face
        shape = predictor(gray, rect)
        shape = shape_to_np(shape)
        # Creating a mask and making only the region with eyes as white
        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        mask = eye_on_mask(mask, left)
        mask = eye_on_mask(mask, right)
        # Image enhancement operations to make the image smoother and easyily detectable
        mask = cv2.dilate(mask, kernel, 5)
        eyes = cv2.bitwise_and(img, img, mask=mask)
        mask = (eyes == [0, 0, 0]).all(axis=2)
        eyes[mask] = [255, 255, 255]
        mid = (shape[42][0] + shape[39][0]) // 2
        eyes_gray = cv2.cvtColor(eyes, cv2.COLOR_BGR2GRAY)
        # For simplicity we assume a standard threshold value we may use gui trackbar by uncommenting the following line
        #threshold = cv2.getTrackbarPos('threshold', 'image')
        threshold = 45
        _, thresh = cv2.threshold(eyes_gray, threshold, 255, cv2.THRESH_BINARY)
        # Image enhacement for the final image as well
        thresh = cv2.erode(thresh, None, iterations=2) #1
        thresh = cv2.dilate(thresh, None, iterations=4) #2
        thresh = cv2.medianBlur(thresh, 3) #3
        thresh = cv2.bitwise_not(thresh)
        contouring(thresh[:, 0:mid], mid, img)
        contouring(thresh[:, mid:], mid, img, True)
        #for (x, y) in shape[36:48]:
        #     cv2.circle(img, (x, y), 2, (255, 0, 0), -1)
    # show the image with the face detections + facial landmarks
    cv2.imshow('eyes', img)
    cv2.imshow("image", thresh)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()